In [1]:
import requests
import pandas as pd
import datetime
from datetime import date
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait 
import time

Telstra_contract_url = "https://www.telstra.com.au/mobile-phones/mobiles-on-a-plan?ti=v3:tcom:corp:pghd:sals:acqs:dflt:adob:othm:defa:mobiles"
Telstra_prepaid_url = 'https://www.telstra.com.au/mobile-phones/prepaid-mobiles/prepaid-phones'
optus_url = 'https://www.optus.com.au/mobile/phones'
optus_prepaid = 'https://www.optus.com.au/prepaid/phones'
vodafone_prepaid = 'https://www.vodafone.com.au/prepaid/mobile-phones'
vodafone_url = 'https://www.vodafone.com.au/mobile/mobile-phones'
urls = [Telstra_contract_url, Telstra_prepaid_url, optus_url, optus_prepaid, vodafone_prepaid, vodafone_url]
urls_o = [vodafone_prepaid]
driver = webdriver.Chrome()


In [2]:
def extract_url_list():
    time.sleep(5)
    new_html = driver.page_source
    soup =BeautifulSoup(new_html, 'html.parser')
    time.sleep(2)
    url_list = []
    sub_url = soup.find_all("a",href=True)
    # extracting all the urls from the site
    for urls in sub_url:
        url_list.append(urls.get('href'))
    return url_list


In [3]:
def extract_price(urls):
    
    telstra = {"Name":[], 'storage':[], "price":[]}
    vodafone = {"Name":[], 'storage':[], "price":[]}
    optus = {"Name":[], 'storage':[], "price":[]}
    
    for url in urls:
        operator_urls = []
        driver.get(url)
        driver.maximize_window()
        time.sleep(1)
        # Telstra Contract
        if url == Telstra_contract_url:
            url_list = extract_url_list()
            # dropping undersired urls
        
            for tel_url in url_list:
                    if ('/mobile-phones/mobiles-on-a-plan/' in tel_url and '#phones' not in tel_url and 
                        "compare-"not in tel_url and 'https://www.telstra.com.au' not in tel_url):
                        operator_urls.append('https://www.telstra.com.au'+tel_url)
            telstra = (Telstra_contract(operator_urls))
            print('Telstra contract extracted')
        # Telstra Prepaid    
        elif url == Telstra_prepaid_url:
            new_html = driver.page_source
            soup =BeautifulSoup(new_html, 'lxml')
            for container in soup.findAll("div", {"class":"lego-device-product-box__info"}):
                telstra['Name'].append(container.find("h3", {"class": "lego-device-product-box__name"}).text.split("\n")[2].strip())
                telstra['price'].append(container.find("div", {"class": "lego-device-product-box__price body-copy-25"}).text[1:])
                telstra['storage'].append('prepaid')
            print('Telstra prepaid data extracted')
        
        # Optus contract
        elif url == optus_url:
            names = {'name':[], 'url':[]}
            optus_urls = []
            for load in range(10):
                try:
                    driver.execute_script("window.scrollTo(0,1700)")
                    driver.find_element(By.XPATH, "//a[contains(text(), 'Load More')]").click()
                except:
                    pass
            driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.CONTROL + Keys.END)
            time.sleep(4)
            url_list = extract_url_list()
         # removing unnecessary urls   
            for tel_url in url_list:
                if('https://www.optus.com.au' in tel_url and'phones/' in tel_url):
                    optus_urls.append(tel_url)
                elif('https://www.optus.com.au/' not in tel_url and '/mobile/phones' in tel_url and 'offer' not in tel_url and 'budget' not in tel_url):
                    optus_urls.append('https://www.optus.com.au'+tel_url)
                elif('offer' in tel_url):
                    optus_urls.append(tel_url)
            check_url = []
            for urls in optus_urls:
                operator_urls.append(((urls.split('?'))[0]))
                operator_urls = list(set(operator_urls))
            operator_urls.remove('https://www.optus.com.au/mobile/phones/budget') 
            operator_urls.remove('https://www.optus.com.au/mobile/phones')
                          
            optus=(optus_data(operator_urls))
            print('optus contract extracted')
            
        # optus prepaid
        elif url == optus_prepaid:
            try:
                driver.execute_script("window.scrollTo(0,1700)")
                driver.find_element(By.XPATH, "//a[contains(text(), 'Load More')]").click()
            except:
                pass
            new_html = driver.page_source
            soup =BeautifulSoup(new_html, 'lxml')
            options = soup.find_all("div", {"class":"lux-component-container",'data-component':"DeviceListing"})
            for container in soup.findAll("div", class_="MobileDevicestyle__MobileDeviceWrapper"):
                optus['Name'].append(container.find("h4").text.strip())
                optus['price'].append(container.find("span", {"class": "price"}).text[1:])
                optus['storage'].append('prepaid')
            print('Optus prepaid data extracted')
                
        elif url == vodafone_prepaid:
            driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.CONTROL + Keys.END)
            new_html = driver.page_source
            soup =BeautifulSoup(new_html, 'lxml')
            options = soup.find_all('div',{'class':'GridColstyles__StyledGridCol-sc-1j4bc83-0 cIAxEE'})
            for product in options:
                vodafone['Name'].append(product.find('div', {"class":"ProductTilestyles__Name-sc-1876jyl-10 ettLiZ"}).text)
                vodafone['price'].append(product.find('span', {"class":'Pricestyles__Amount-sc-1m965dg-9 dmrNCW'}).text)
                vodafone['storage'].append('prepaid')
            print('vodafone prepaid data extracted')        
            
        # Vodafone     
        else:
            driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.CONTROL + Keys.END)
            time.sleep(3)
            url_list = extract_url_list()
            for tel_url in url_list:
                if('//www.vodafone.com.au/' in tel_url):
                    operator_urls.append(tel_url)
                elif('mobile-phones' in tel_url and 'Referrer' not in tel_url and 'https//www.vodafone.com.au/mobile/mobile-phones/apple' not in tel_url):
                    operator_urls.append('https://www.vodafone.com.au'+tel_url)
                
                
                    
            vodafone = (vodafone_data(operator_urls))
            print('vodafone contract extracted')
        
        
    return telstra, optus, vodafone  


In [4]:
# function to extract name, storage, and price of different models from Telstra site
def Telstra_contract(telstra_urls):
    table={"Name":[], 'storage':[], "price":[]}
    for url in telstra_urls:
        driver.get(url)
        time.sleep(2)
        # finding available storages
        driver.execute_script("window.scrollTo(0,500)")
        try:
            storage_capacities = driver.find_element(By.XPATH,'//*[@id="main-content"]/div[1]/div[1]/article/div/div[4]/form/fieldset[2]/div').text.split()
        except:
            storage_capacities = driver.find_element(By.XPATH,'//*[@id="main-content"]/div[2]/div[1]/article/div/div[4]/form/fieldset[2]/div').text.split()
        container = driver.find_element(By.XPATH,"//div[@class='device-product__butons-container']")
        options = container.find_elements(By.XPATH, ".//div[@class='device-product__button-option']")
        if len(storage_capacities) == 1:
            table['Name'].append(url.split("/")[len(url.split("/"))-1].replace("-"," "))
            table['storage'].append(storage_capacities[0])
            time.sleep(1)
            try:
                table['price'].append(float(driver.find_element(By.XPATH,'//*[@id="main-content"]/div[1]/div[1]/article/div/div[4]/form/div[4]/div[1]/span/').text[1:6].strip())*36)
            except:
                try:
                    table['price'].append(float(driver.find_element(By.XPATH,'//*[@id="main-content"]/div[1]/div[1]/article/div/div[4]/div[2]/div[1]/span').text[1:6].strip())*36)
                except:
                    try:
                        table['price'].append(float(driver.find_element(By.XPATH,'//*[@id="main-content"]/div[1]/div[1]/article/div/div[4]/form/div[4]/div[1]/span/span[1]').text[1:6].strip())*36) 
                    except:
                        table['price'].append(float(driver.find_element(By.XPATH,'//*[@id="main-content"]/div[2]/div[1]/article/div/div[4]/div[2]/div[1]/span').text[1:6].strip())*36)

        else:
            table['Name'].append(url.split("/")[len(url.split("/"))-1].replace("-"," "))
            table['storage'].append(storage_capacities[0])
            time.sleep(1)
            try:
                table['price'].append(float(driver.find_element(By.XPATH,'//*[@id="main-content"]/div[1]/div[1]/article/div/div[4]/div[2]/div[1]/span').text[1:6].strip())*36)
            except:
                table['price'].append(float(driver.find_element(By.XPATH,'//*[@id="main-content"]/div[2]/div[1]/article/div/div[4]/div[2]/div[1]/span').text[1:6].strip())*36)
            for stcap in storage_capacities[:0:-1]:
    #             if len(storage_capacities) != len(options) and len(storage_capacities)!=1:
                    for option in options:
                        # button lable value
                        op = option.get_attribute("data-mobile-product-storage-value")
                        if op == stcap:
                            time.sleep(1)
                            option.click()
                            time.sleep(1)
                            table['Name'].append(url.split("/")[len(url.split("/"))-1].replace("-"," "))
                            table['storage'].append(op)
                            try:
                                table['price'].append(float(driver.find_element(By.XPATH,'//*[@id="main-content"]/div[1]/div[1]/article/div/div[4]/div[2]/div[1]/span')
                                                                    .text[1:6].strip())*36)
                            except:
                                table['price'].append(float(driver.find_element(By.XPATH,'//*[@id="main-content"]/div[2]/div[1]/article/div/div[4]/div[2]/div[1]/span/span[1]').text[1:6].strip())*36)
    
    return table


In [5]:
# function to extract name, storage, and price of different models from Vodafone site
def vodafone_data(vodafone_urls):
    vodafone = {"Name":[], 'storage':[], "price":[]}
    for url in vodafone_urls:
        driver.get(url)
        time.sleep(1)
        # finding available storages
        
        new_html = driver.page_source
        soup =BeautifulSoup(new_html, 'lxml') 
        capacities = []
        labels = soup.select(".OptionButtonGroupstyles__OptionButtonGroup-sc-1t2rvjr-0 label")
# Extract the text (storage capacity values) from the labels
        for label in labels:
            if ('months' not in label.get_text()):
                capacities.append(label.get_text())
        if len(capacities) == 1:
            vodafone['Name'].append(driver.find_element(By.XPATH,'//*[@id="__next"]/div[2]/main/section[1]/div[1]/div/div/div/h1').text)
            vodafone['storage'].append(capacities[0]) 
            vodafone['price'].append(float(driver.find_element(By.CLASS_NAME, "Pricestyles__Amount-sc-1m965dg-9").text)*36)
        else:
            for options in capacities:
                url2 = url+'?capacity='+options
                driver.get(url2)
                vodafone['Name'].append(driver.find_element(By.XPATH,'//*[@id="__next"]/div[2]/main/section[1]/div[1]/div/div/div/h1').text)
                vodafone['storage'].append(options) 
                vodafone['price'].append(float(driver.find_element(By.CLASS_NAME, "Pricestyles__Amount-sc-1m965dg-9").text)*36)   
    
    return vodafone
            

In [6]:
def click_radio_input(input_element):
    driver.execute_script("arguments[0].click();", input_element)
def optus_data(optus_urls):
    optus = {"Name":[], 'storage':[], "price":[]}
    for url in optus_urls:
        storage_capacity = []
        driver.get(url)
        time.sleep(1)
        # finding available storages 
        new_html = driver.page_source
        soup =BeautifulSoup(new_html, 'lxml') 
        driver.execute_script("window.scrollTo(0,300)")
        time.sleep(1)
        storage_elements = soup.select(".sc-hZgfyJ.cNXwBI.radio-label")
        for elem in storage_elements:
            storage_options = elem.text.strip()
            if storage_options.endswith('B'):
                storage_capacity.append(storage_options)
        if len(storage_capacity) == 1:
            new_html = driver.page_source
            soup =BeautifulSoup(new_html, 'lxml') 
            optus['Name'].append(soup.find('div', {'class': 'lux-component-container','data-component':"ProductText"}).text)
            optus['storage'].append(storage_capacity[0])
            try:
                optus['price'].append(float(soup.find('div', {'class': 'amount'}).text[1:].strip())*36)
            except:
                try:
                    optus['price'].append(float(soup.find('div', {'class': 'lux-component-container','data-component':"ProductPrice"}).text[1:].strip('per month'))*36)        
                except:
                    try:
                        optus['price'].append(float(driver.find_element(By.XPATH, '//*[@id="par_responsive_container_responsive_container_746448112_sd_component_mec_pro_780928758_140976078"]/div/div/div/div[1]')
                                                    .text[1:])*36)
                        
                    except:
                        optus['price'].append('Check: '+url)
                    
                    
                
        else:
            storage_options = driver.find_elements(By.NAME, 'CapacitySwatch')
            for cap in storage_options[::-1]:
                click_radio_input(cap)
                time.sleep(1)
                new_html = driver.page_source
                soup =BeautifulSoup(new_html, 'lxml') 
                try:
                    optus['Name'].append(soup.find('div', {'class': 'lux-component-container','data-component':"ProductText"}).text)
                except:
                    optus['Name'].append(soup.find('h1', {'class': 'product-title-desktop d-none d-md-block'}).text)
                        
                capacity_swatch_div = soup.find('div', class_='CapacitySwatchstyle__Swatches')

                # Find the input element with "aria-checked='true'" (the selected storage option)
                try:
                    selected_input = capacity_swatch_div.find('input', {'aria-checked': 'true'})
                    optus['storage'].append(selected_input['value'])
                except:
                    selected_input = (cap.text)
                    optus['storage'].append(selected_input)
                

                # Get the value of the selected storage option from the 'value' attribute of the input element
               

                try:
                    optus['price'].append(float(soup.find('div', {'class': 'amount'}).text[1:].strip())*36)
                except:
                    try:
                        optus['price'].append(float(soup.find('div', {'class': 'lux-component-container','data-component':"ProductPrice"}).text[1:].strip('per month'))*36)
                    except:
                        try:
                            optus['price'].append(float(driver.find_element(By.XPATH, '//*[@id="par_responsive_container_responsive_container_746448112_sd_component_mec_pro_780928758_140976078"]/div/div/div/div[1]')
                                                    .text[1:])*36)
                        
                        except:
                            optus['price'].append('Check: '+url)
                    
                        
                        
                        
                        
    return optus
    
    

In [7]:
def generate_csv():
    telstra, optus, vodafone = extract_price(urls)
    telstra_df = pd.DataFrame(telstra)
    optus_df = pd.DataFrame(optus)
    vodafone_df = pd.DataFrame(vodafone)
    period = pd.read_csv('Period_mapping.csv')
    week = 'default'
    for i in range(period.shape[0]):
        if  pd.to_datetime(period['Start Date'][i],dayfirst=True) <= datetime.date.today() <= pd.to_datetime(period['End Date'][i],dayfirst=True):
            week = str(period['Period_Mapping'][i])
    telstra_df.to_csv('telstra'+'_'+week+'.CSV')
    optus_df.to_csv('optus'+'_'+week+'.CSV')
    vodafone_df.to_csv('vodafone'+'_'+week+'.CSV')
    
        

In [8]:
generate_csv()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main-content"]/div[2]/div[1]/article/div/div[4]/form/fieldset[2]/div"}
  (Session info: chrome=115.0.5790.110); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x003EA813+48355]
	(No symbol) [0x0037C4B1]
	(No symbol) [0x00285358]
	(No symbol) [0x002B09A5]
	(No symbol) [0x002B0B3B]
	(No symbol) [0x002DE232]
	(No symbol) [0x002CA784]
	(No symbol) [0x002DC922]
	(No symbol) [0x002CA536]
	(No symbol) [0x002A82DC]
	(No symbol) [0x002A93DD]
	GetHandleVerifier [0x0064AABD+2539405]
	GetHandleVerifier [0x0068A78F+2800735]
	GetHandleVerifier [0x0068456C+2775612]
	GetHandleVerifier [0x004751E0+616112]
	(No symbol) [0x00385F8C]
	(No symbol) [0x00382328]
	(No symbol) [0x0038240B]
	(No symbol) [0x00374FF7]
	BaseThreadInitThunk [0x76CF7D59+25]
	RtlIni